In [9]:
import cv2
import numpy as np
import face_recognition
import os
from datetime import datetime
import string
import random
import time


In [ ]:
# count = len([name for name in os.listdir(DIR) if name !='.DS_Store'])

In [2]:
path = 'images'
images = []
personNames = []
myList = ([ex for ex in os.listdir(path) if ex !=".DS_Store"])
print(myList)

['Matthew.jpg', 'Steve Carell.jpg', 'Tom Hanks.jpg']


In [3]:
for cu_img in myList:
    current_Img = cv2.imread(f'{path}/{cu_img}')
    images.append(current_Img)
    personNames.append(os.path.splitext(cu_img)[0])
print(personNames)

['Matthew', 'Steve Carell', 'Tom Hanks']


In [4]:
def faceEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encode = face_recognition.face_encodings(img)[0]
        encodeList.append(encode)
    return encodeList

In [10]:
def id_generator(size=6, chars=string.ascii_uppercase + string.digits):
    boardingNumber = ''.join(random.choice(chars) for _ in range(size))
    
    return boardingNumber

In [11]:
def passInfo(name):
    with open('Boarding.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        count = 0
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
            count = count+1
        if name not in nameList:
            
            time_now = datetime.now()
            tStr = time_now.strftime('%H:%M:%S')
            dStr = time_now.strftime('%d/%m/%Y')
            bStr = id_generator()
            count = len(nameList)
            
            
            
            f.writelines(f'\n{name}, {tStr}, {dStr}, {bStr}, {count}')
                

In [6]:
encodeListKnown = faceEncodings(images)
print('All Encodings Complete!!!')

All Encodings Complete!!!


In [15]:
cap = cv2.VideoCapture(0)

opc_count = 0
object_id_list = []
dtime = dict()
dwell_time = dict()

while True:
    ret, frame = cap.read()
    faces = cv2.resize(frame, (0, 0), None, 0.25, 0.25)
    faces = cv2.cvtColor(faces, cv2.COLOR_BGR2RGB)

    facesCurrentFrame = face_recognition.face_locations(faces)
    encodesCurrentFrame = face_recognition.face_encodings(faces, facesCurrentFrame)
    i=0
    for encodeFace, faceLoc in zip(encodesCurrentFrame, facesCurrentFrame):
        matches = face_recognition.compare_faces(encodeListKnown, encodeFace)
        faceDis = face_recognition.face_distance(encodeListKnown, encodeFace)
        # print(faceDis)
        matchIndex = np.argmin(faceDis)

        if matches[matchIndex]:
            name = personNames[matchIndex].upper()
            # print(name)
            y1, x2, y2, x1 = faceLoc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4
            cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 255, 0), 2)
            cv2.rectangle(frame, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            
            if matchIndex not in object_id_list:
                object_id_list.append(matchIndex)
                dtime[matchIndex] = datetime.now()
                dwell_time[matchIndex] = 0
            else:
                curr_time = datetime.now()
                old_time = dtime[matchIndex]
                time_diff = curr_time - old_time
                dtime[matchIndex] = datetime.now()
                sec = time_diff.total_seconds()
                dwell_time[matchIndex] += sec
            i = i+1
            cv2.putText(frame, name, (x1 + 6, y2 - 6), cv2.FONT_HERSHEY_DUPLEX, 0.5, (255, 255, 255), 2)
            cv2.putText(frame, "Face" + str(i),(x1-13,y1-12), cv2.FONT_HERSHEY_DUPLEX, 0.7, (0,0,255), 2)
            text = 'Time: {}'.format(int(dwell_time[matchIndex]))
            cv2.putText(frame, text, (x2+2, y1-10), cv2.FONT_HERSHEY_DUPLEX, 0.7, (255,255,255), 2)            
            passInfo(name)
            
            if matchIndex not in object_id_list:
                object_id_list.append(matchIndex)
    opc_count = len(object_id_list)
    opc_text = ("Footfall:{}".format(opc_count))
    cv2.putText(frame, opc_text,(5,30), cv2.FONT_HERSHEY_DUPLEX, 1, (255,255,255), 2)
    
    cv2.imshow('Webcam', frame)
    
    if cv2.waitKey(1) == 13:
        break

cap.release()
cv2.destroyAllWindows()